README - IMPORTANTTTT

Import Libraries

`python -m pip install pymongo[srv]`

additionally, please download https://dev.mysql.com/doc/index-other.html, 

Specifically, the sakila database!!! Canvas would not allow me to attach .sql files?

The other database is "sample_mflix" in MongoDB (one of the free samples that you can add to clusters)

In total, used 4 data-sets,
Northwind_DW (for dim_date),
Sample_mflix (from MongoDB),
Sakila database (from mySQL link),
Attached Top_1000_IMBD.csv


In [10]:
import os
import json
import numpy
import datetime
import certifi
import pandas as pd

import pymongo
import sqlalchemy
from sqlalchemy import create_engine, text

print(f"Running SQL Alchemy Version: {sqlalchemy.__version__}")
print(f"Running PyMongo Version: {pymongo.__version__}")

Running SQL Alchemy Version: 2.0.45
Running PyMongo Version: 4.15.4


#### Declare & Assign Connection Variables for the MongoDB Server, the MySQL Server & Databases with which You'll be Working 

In [11]:
mysql_args = {
    "uid": "root",
    "pwd" : "Weiao123.1",  #password removed for privacy
    "hostname": "localhost",
    "dbname": "sakila"   
}

# The 'cluster_location' must either be "atlas" or "local".
mongodb_args = {
    "user_name" : "patrickyin3",
    "password" : "patrickyin3",
    "cluster_name" : "ds2002",
    "cluster_subnet" : "whjs4xu",
    "cluster_location" : "atlas", # "local"
    "db_name" : "sample_mflix"  #USED SAMPLE_MFLIX IN MONGODB (free dataset)
}

mysql_args_nw = {
    "uid": "root",
    "pwd" : "Weiao123.1",  #password removed for privacy
    "hostname": "localhost",
    "dbname": "northwind_dw" 
}


Define Functions for Getting Data From and Setting Data Into Databases

In [12]:
def get_sql_dataframe(sql_query, **args):
    '''Create a connection to the MySQL database'''
    conn_str = f"mysql+pymysql://{args['uid']}:{args['pwd']}@{args['hostname']}/{args['dbname']}"
    sqlEngine = create_engine(conn_str, pool_recycle=3600)
    connection = sqlEngine.connect()
    
    '''Invoke the pd.read_sql() function to query the database, and fill a Pandas DataFrame.'''
    dframe = pd.read_sql(text(sql_query), connection);
    connection.close()
    
    return dframe
    

def set_dataframe(df, table_name, pk_column, db_operation, **args):
    '''Create a connection to the MySQL database'''
    conn_str = f"mysql+pymysql://{args['uid']}:{args['pwd']}@{args['hostname']}/{args['dbname']}"
    sqlEngine = create_engine(conn_str, pool_recycle=3600)
    connection = sqlEngine.connect()
    
    '''Invoke the Pandas DataFrame .to_sql( ) function to either create, or append to, a table'''
    if db_operation == "insert":
        df.to_sql(table_name, con=connection, index=False, if_exists='replace')
        connection.execute(text(f"ALTER TABLE {table_name} ADD PRIMARY KEY ({pk_column});"))
            
    elif db_operation == "update":
        df.to_sql(table_name, con=connection, index=False, if_exists='append')
    
    connection.close()


def get_mongo_client(**args):
    '''Validate proper input'''
    if args["cluster_location"] not in ['atlas', 'local']:
        raise Exception("You must specify either 'atlas' or 'local' for the cluster_location parameter.")
    
    else:
        if args["cluster_location"] == "atlas":
            connect_str = f"mongodb+srv://{args['user_name']}:{args['password']}@"
            connect_str += f"{args['cluster_name']}.{args['cluster_subnet']}.mongodb.net"
            client = pymongo.MongoClient(connect_str, tlsCAFile=certifi.where())
            
        elif args["cluster_location"] == "local":
            client = pymongo.MongoClient("mongodb://localhost:27017/")
        
    return client


def get_mongo_dataframe(mongo_client, db_name, collection, query):
    '''Query MongoDB, and fill a python list with documents to create a DataFrame'''
    db = mongo_client[db_name]
    dframe = pd.DataFrame(list(db[collection].find(query)))
    dframe.drop(['_id'], axis=1, inplace=True)
    mongo_client.close()
    
    return dframe


def set_mongo_collections(mongo_client, db_name, data_directory, json_files):
    db = mongo_client[db_name]
    
    for file in json_files:
        db.drop_collection(file)
        json_file = os.path.join(data_directory, json_files[file])
        with open(json_file, 'r') as openfile:
            json_object = json.load(openfile)
            file = db[file]
            result = file.insert_many(json_object)
        
    mongo_client.close()

### Load Date Dimension Data From Source Database


In [13]:
sql_date = "SELECT * FROM dim_date;"
df_dim_date = get_sql_dataframe(sql_date, **mysql_args_nw)

# Keep only relevant columns
cols_date = ['date_key', 'full_date', 'day_of_week', 'day_name_of_week',
             'day_of_month', 'day_of_year', 'weekday_weekend', 'month_of_year', 
             'calendar_year', 'calendar_quarter', 'fiscal_year', 'fiscal_quarter']

df_dim_date = df_dim_date[[c for c in cols_date if c in df_dim_date.columns]]

set_dataframe(df_dim_date, table_name='dim_date', pk_column='date_key', db_operation='insert', **mysql_args)

df_dim_date.head(5)

,date_key,full_date,day_of_week,day_name_of_week,day_of_month,day_of_year,weekday_weekend,month_of_year,calendar_year,calendar_quarter,fiscal_year,fiscal_quarter
0,20000101,2000-01-01,7,Saturday,1,1,Weekend,1,2000,1,2000,3
1,20000102,2000-01-02,1,Sunday,2,2,Weekend,1,2000,1,2000,3
2,20000103,2000-01-03,2,Monday,3,3,Weekday,1,2000,1,2000,3
3,20000104,2000-01-04,3,Tuesday,4,4,Weekday,1,2000,1,2000,3
4,20000105,2000-01-05,4,Wednesday,5,5,Weekday,1,2000,1,2000,3


### Import IMDb Top 1000 Dataset From Local CSV File


In [14]:
#import imdb_top_1000.csv from local

import os
import pandas as pd


base_dir = os.path.dirname(os.path.abspath("__file__"))  
csv_path = os.path.join(base_dir, "imdb_top_1000.csv")

df_imdb = pd.read_csv(csv_path)
df_imdb.head(3)


,Poster_Link,Series_Title,Released_Year,Certificate,Runtime,Genre,IMDB_Rating,Overview,Meta_score,Director,Star1,Star2,Star3,Star4,No_of_Votes,Gross
0,https://m.media-amazon.com/images/M/MV5BMDFkYT...,The Shawshank Redemption,1994,A,142 min,Drama,9.3,Two imprisoned men bond over a number of years...,80.0,Frank Darabont,Tim Robbins,Morgan Freeman,Bob Gunton,William Sadler,2343110,"28,341,469"
1,https://m.media-amazon.com/images/M/MV5BM2MyNj...,The Godfather,1972,A,175 min,"Crime, Drama",9.2,An organized crime dynasty's aging patriarch t...,100.0,Francis Ford Coppola,Marlon Brando,Al Pacino,James Caan,Diane Keaton,1620367,"134,966,411"
2,https://m.media-amazon.com/images/M/MV5BMTMxNT...,The Dark Knight,2008,UA,152 min,"Action, Crime, Drama",9.0,When the menace known as the Joker wreaks havo...,84.0,Christopher Nolan,Christian Bale,Heath Ledger,Aaron Eckhart,Michael Caine,2303232,"534,858,444"


### Transform IMDb Dataset and Load IMDb Dimension Table


In [15]:
# Keep relevant columns and rename
cols_imdb = ['Series_Title', 'Released_Year', 'Certificate', 'Runtime', 'Genre',
             'IMDB_Rating', 'Meta_score', 'Overview', 'Director', 'Star1', 'Star2', 'Star3', 'Star4', 'No_of_Votes', 'Gross']
df_imdb = df_imdb[[c for c in cols_imdb if c in df_imdb.columns]]

df_imdb.rename(columns={
    'Series_Title': 'title',
    'Released_Year': 'year',
    'IMDB_Rating': 'imdb_rating',
    'Meta_score': 'metascore',
    'No_of_Votes': 'imdb_votes',
    'Gross': 'gross',
    'Star1': 'cast_1',
    'Star2': 'cast_2',
    'Star3': 'cast_3',
    'Star4': 'cast_4'
}, inplace=True)

# Add surrogate key
df_imdb.insert(0, 'movie_key', range(1, df_imdb.shape[0]+1))

# Keep final columns for MySQL
df_dim_imdb = df_imdb[['movie_key', 'title', 'year', 'Certificate', 'Runtime', 'Genre',
                        'Director', 'cast_1', 'cast_2', 'cast_3', 'cast_4', 'imdb_rating',
                        'metascore', 'imdb_votes', 'gross', 'Overview']]

# Insert into MySQL
set_dataframe(df_dim_imdb, table_name='dim_imdb_top_1000', pk_column='movie_key', db_operation='insert', **mysql_args)

# Verify
df_imdb_mysql = get_sql_dataframe("SELECT * FROM dim_imdb_top_1000 LIMIT 5;", **mysql_args)
df_imdb_mysql.head(5)

,movie_key,title,year,Certificate,Runtime,Genre,Director,cast_1,cast_2,cast_3,cast_4,imdb_rating,metascore,imdb_votes,gross,Overview
0,1,The Shawshank Redemption,1994,A,142 min,Drama,Frank Darabont,Tim Robbins,Morgan Freeman,Bob Gunton,William Sadler,9.3,80.0,2343110,"28,341,469",Two imprisoned men bond over a number of years...
1,2,The Godfather,1972,A,175 min,"Crime, Drama",Francis Ford Coppola,Marlon Brando,Al Pacino,James Caan,Diane Keaton,9.2,100.0,1620367,"134,966,411",An organized crime dynasty's aging patriarch t...
2,3,The Dark Knight,2008,UA,152 min,"Action, Crime, Drama",Christopher Nolan,Christian Bale,Heath Ledger,Aaron Eckhart,Michael Caine,9.0,84.0,2303232,"534,858,444",When the menace known as the Joker wreaks havo...
3,4,The Godfather: Part II,1974,A,202 min,"Crime, Drama",Francis Ford Coppola,Al Pacino,Robert De Niro,Robert Duvall,Diane Keaton,9.0,90.0,1129952,"57,300,000",The early life and career of Vito Corleone in ...
4,5,12 Angry Men,1957,U,96 min,"Crime, Drama",Sidney Lumet,Henry Fonda,Lee J. Cobb,Martin Balsam,John Fiedler,9.0,96.0,689845,"4,360,000",A jury holdout attempts to prevent a miscarria...


### Transform MongoDB Movie Data and Load Movies Dimension Table


In [16]:
import numpy as np
import pandas as pd

# DIM MOVIES (MongoDB)

client = get_mongo_client(**mongodb_args)
df_movies = get_mongo_dataframe(client, mongodb_args["db_name"], "embedded_movies", {})
client.close()

# Rename primary key and keep relevant columns
df_movies.rename(columns={'_id': 'movie_id'}, inplace=True)
cols_movies = ['movie_id', 'title', 'year', 'runtime', 'genres', 'directors', 'cast', 'metacritic']
df_movies = df_movies[[c for c in cols_movies if c in df_movies.columns]]

# Convert list columns to comma-separated strings
for col in ['genres', 'directors', 'cast']:
    if col in df_movies.columns:
        df_movies[col] = df_movies[col].apply(lambda x: ', '.join(x) if isinstance(x, list) else x)

# Add surrogate key
df_movies.insert(0, 'movie_key', range(1, df_movies.shape[0]+1))

# Keep only dimension columns
df_dim_movies = df_movies[['movie_key', 'title', 'year', 'runtime', 'genres', 'directors', 'cast', 'metacritic']]

# Insert into MySQL
set_dataframe(df_dim_movies, table_name='dim_movies', pk_column='movie_key', db_operation='insert', **mysql_args)

# Verify
df_movies_mysql = get_sql_dataframe("SELECT * FROM dim_movies LIMIT 5;", **mysql_args)
df_movies_mysql.head(5)


,movie_key,title,year,runtime,genres,directors,cast,metacritic
0,1,Men Without Women,1930,77.0,"Action, Drama",John Ford,"Kenneth MacKenna, Frank Albertson, J. Farrell ...",None
1,2,For Heaven's Sake,1926,58.0,"Action, Comedy, Romance",Sam Taylor,"Harold Lloyd, Jobyna Ralston, Noah Young, Jim ...",None
2,3,The Mark of Zorro,1940,94.0,"Action, Adventure, Romance",Rouben Mamoulian,"Tyrone Power, Linda Darnell, Basil Rathbone, G...",None
3,4,Beau Geste,1926,101.0,"Action, Adventure, Drama",Herbert Brenon,"Ronald Colman, Neil Hamilton, Ralph Forbes, Al...",None
4,5,North West Mounted Police,1940,126.0,"Action, Adventure, Drama",Cecil B. DeMille,"Gary Cooper, Madeleine Carroll, Paulette Godda...",None


### Extract User Data From MongoDB and Load Users Dimension Table


In [17]:
# DIM USERS (MongoDB)

client = get_mongo_client(**mongodb_args)
df_users = get_mongo_dataframe(client, mongodb_args["db_name"], "users", {})
client.close()

# Rename primary key and keep relevant columns
df_users.rename(columns={'_id': 'user_id'}, inplace=True)
cols_users = ['user_id', 'name', 'email']
df_users = df_users[[c for c in cols_users if c in df_users.columns]]

# Add surrogate key
df_users.insert(0, 'user_key', range(1, df_users.shape[0]+1))

# Keep only dimension columns
df_dim_users = df_users[['user_key', 'name', 'email']]

# Insert into MySQL
set_dataframe(df_dim_users, 'dim_users', 'user_key', 'insert', **mysql_args)

# Verify
df_users_mysql = get_sql_dataframe("SELECT * FROM dim_users LIMIT 5;", **mysql_args)
df_users_mysql.head(5)

,user_key,name,email
0,1,Bran Stark,isaac_hempstead_wright@gameofthron.es
1,2,Sansa Stark,sophie_turner@gameofthron.es
2,3,Sandor Clegane,rory_mccann@gameofthron.es
3,4,Jaime Lannister,nikolaj_coster-waldau@gameofthron.es
4,5,Catelyn Stark,michelle_fairley@gameofthron.es


### Extract Comment Data From MongoDB and Load Comments Fact Table


In [18]:
# FACT COMMENTS (MongoDB)

client = get_mongo_client(**mongodb_args)
df_comments = get_mongo_dataframe(client, mongodb_args["db_name"], "comments", {})
client.close()

# Rename primary key and keep relevant columns
df_comments.rename(columns={'_id': 'comment_id'}, inplace=True)
cols_comments = ['comment_id', 'text', 'date', 'movie_id', 'user_id']
df_comments = df_comments[[c for c in cols_comments if c in df_comments.columns]]

# Add surrogate key
df_comments.insert(0, 'comment_key', range(1, df_comments.shape[0]+1))

# Map user_key and movie_key 
df_comments['user_key'] = np.random.choice(df_dim_users['user_key'], size=len(df_comments))
df_comments['movie_key'] = np.random.choice(df_dim_movies['movie_key'], size=len(df_comments))

# Keep only fact table columns
df_fact_comments = df_comments[['comment_key', 'movie_key', 'user_key', 'text', 'date']]

# Insert into MySQL
set_dataframe(df_fact_comments, 'fact_comments', 'comment_key', 'insert', **mysql_args)

# Verify
df_fact_comments_mysql = get_sql_dataframe("SELECT * FROM fact_comments LIMIT 5;", **mysql_args)
df_fact_comments_mysql.head(5)



,comment_key,movie_key,user_key,text,date
0,1,1939,72,Hic corporis illo adipisci similique. Omnis te...,2006-03-07 07:15:16
1,2,3380,170,Nobis incidunt ea tempore cupiditate sint. Ita...,2012-11-26 11:00:57
2,3,1982,182,Voluptatum voluptatem nam et accusamus ullam q...,2015-02-08 01:28:23
3,4,1178,185,Illo nihil occaecati minima facere ea nemo. Mo...,1976-12-18 08:14:46
4,5,2206,16,Minima odit officiis minima nam. Aspernatur id...,1981-11-08 04:32:25


### Extract Sakila Data and Load Actor, Film, and Film-Actor Tables


In [19]:
# SAKILA ETL (MySQL)

#DIM ACTOR
df_actor = get_sql_dataframe("SELECT * FROM actor;", **mysql_args)
df_actor.insert(0, 'actor_key', range(1, len(df_actor)+1))
df_dim_actor = df_actor[['actor_key', 'first_name', 'last_name']]
set_dataframe(df_dim_actor, 'dim_actor', 'actor_key', 'insert', **mysql_args)

#DIM FILM
df_film = get_sql_dataframe("SELECT * FROM film;", **mysql_args)
df_film.insert(0, 'film_key', range(1, len(df_film)+1))
df_dim_film = df_film[['film_key', 'title', 'release_year', 'rating', 'length']]
set_dataframe(df_dim_film, 'dim_film', 'film_key', 'insert', **mysql_args)

#FACT FILM_ACTOR
df_film_actor = get_sql_dataframe("SELECT * FROM film_actor;", **mysql_args)
film_map = dict(zip(df_film['film_id'], df_dim_film['film_key']))
actor_map = dict(zip(df_actor['actor_id'], df_dim_actor['actor_key']))

df_film_actor['film_key'] = df_film_actor['film_id'].map(film_map)
df_film_actor['actor_key'] = df_film_actor['actor_id'].map(actor_map)

df_fact_film_actor = df_film_actor[['film_key', 'actor_key']]
df_fact_film_actor.insert(0, 'film_actor_key', range(1, len(df_fact_film_actor)+1))

set_dataframe(df_fact_film_actor, 'fact_film_actor', 'film_actor_key', 'insert', **mysql_args)

# Verify
df_fact_film_actor_mysql = get_sql_dataframe("SELECT * FROM fact_film_actor LIMIT 5;", **mysql_args)
df_fact_film_actor_mysql.head(5)

,film_actor_key,film_key,actor_key
0,1,1,1
1,2,23,1
2,3,25,1
3,4,106,1
4,5,140,1


### Query Top 5 User–Movie Combinations by Total Comments


In [20]:
# QUERY 1: Total Comments per User per Movie 
# This query joins fact_comments, dim_users, and dim_movies to find how many comments each user has made per movie. It shows the top 5 user–movie combinations with the most comments.

sql_comments_per_user_movie = """
SELECT
    u.user_key,
    u.name AS user_name,
    m.movie_key,
    m.title AS movie_title,
    COUNT(c.comment_key) AS total_comments
FROM
    fact_comments c
JOIN
    dim_users u ON c.user_key = u.user_key
JOIN
    dim_movies m ON c.movie_key = m.movie_key
GROUP BY
    u.user_key, u.name, m.movie_key, m.title
ORDER BY
    total_comments DESC
LIMIT 20;
"""

df_comments_per_user_movie = get_sql_dataframe(sql_comments_per_user_movie, **mysql_args)
df_comments_per_user_movie.head(5)


,user_key,user_name,movie_key,movie_title,total_comments
0,14,Cersei Lannister,2622,Death Becomes Her,3
1,161,Jennifer Frazier,814,"A Fairly Odd Movie: Grow Up, Timmy Turner!",3
2,156,Bradley Brooks,200,Cop or Hood,3
3,94,Jojen Reed,100,The Guns of Navarone,3
4,126,Denise Bryant,50,Triple Cross,3


### Compute Average IMDb Rating per User Based on Their Comments


In [21]:
#QUERY 2: Average IMDb Rating per User
# This query computes the average IMDb rating of all movies commented on by each user. It joins fact_comments, dim_users, and dim_imdb_top_1000 and returns the top 5 users ranked by their average movie rating.

sql_avg_rating_per_user = """
SELECT
    u.user_key,
    u.name AS user_name,
    COUNT(c.comment_key) AS total_comments,
    AVG(m.imdb_rating) AS avg_imdb_rating
FROM
    fact_comments c
JOIN
    dim_users u ON c.user_key = u.user_key
JOIN
    dim_imdb_top_1000 m ON c.movie_key = m.movie_key
GROUP BY
    u.user_key, u.name
ORDER BY
    avg_imdb_rating DESC
LIMIT 20;
"""

df_avg_rating_per_user = get_sql_dataframe(sql_avg_rating_per_user, **mysql_args)
df_avg_rating_per_user.head(5)


,user_key,user_name,total_comments,avg_imdb_rating
0,99,Javier Smith,77,8.029870
1,59,Lancel Lannister,58,8.020690
2,143,Andrea Le,67,8.016418
3,126,Denise Bryant,69,8.011594
4,13,Ned Stark,73,8.010959


### Analyze Sakila Customers by Total Comments and Average IMDb Ratings


In [22]:
#QUERY 3: Comments and Average Ratings by Sakila Customers
# This query integrates the Sakila database by linking users to Sakila customers (matching by user_key). It shows, for each customer, the total number of comments and the average IMDb rating of the movies they commented on.

sql_comments_sakila = """
SELECT
    cust.customer_id,
    cust.first_name,
    cust.last_name,
    COUNT(c.comment_key) AS total_comments,
    AVG(m.imdb_rating) AS avg_movie_rating
FROM
    fact_comments c
JOIN
    dim_users u ON c.user_key = u.user_key
JOIN
    dim_imdb_top_1000 m ON c.movie_key = m.movie_key
JOIN
    sakila.customer cust ON cust.customer_id = u.user_key
GROUP BY
    cust.customer_id, cust.first_name, cust.last_name
ORDER BY
    total_comments DESC
LIMIT 20;
"""

df_comments_sakila = get_sql_dataframe(sql_comments_sakila, **mysql_args)
df_comments_sakila.head(5)


,customer_id,first_name,last_name,total_comments,avg_movie_rating
0,152,ALICIA,MILLS,86,7.937209
1,128,MARJORIE,TUCKER,84,7.980952
2,86,JACQUELINE,LONG,83,7.902410
3,110,TIFFANY,JORDAN,82,7.985366
4,177,SAMANTHA,DUNCAN,82,7.941463


In [1]:
import os, shutil, time

def remove_directory_tree(path: str):
    if os.path.exists(path):
        shutil.rmtree(path)

def wait_until_stream_is_ready(query, min_batches=1):
    while len(query.recentProgress) < min_batches:
        time.sleep(5)
    print(f"Stream processed {len(query.recentProgress)} batches")


In [4]:
import os
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, current_timestamp, input_file_name
from pyspark.sql.types import LongType, IntegerType
from pyspark.sql.types import StringType, StructType, StructField, LongType


BASE_DIR = os.path.join(os.getcwd(), "lakehouse")

BRONZE_DIR = os.path.join(BASE_DIR, "bronze")
SILVER_DIR = os.path.join(BASE_DIR, "silver")
GOLD_DIR   = os.path.join(BASE_DIR, "gold")

remove_directory_tree(BASE_DIR)
os.makedirs(BRONZE_DIR)
os.makedirs(SILVER_DIR)
os.makedirs(GOLD_DIR)



In [5]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("DS-2002 Final Project") \
    .master("local[*]") \
    .config("spark.sql.streaming.schemaInference", "true") \
    .config("spark.sql.shuffle.partitions", "4") \
    .getOrCreate()

spark.sparkContext.setLogLevel("ERROR")


FileNotFoundError: [WinError 2] The system cannot find the file specified

In [143]:
df_comments_spark.write.mode("overwrite").parquet(bronze_path_comments)
print("Bronze Layer sample:")
spark.read.schema(comments_schema).parquet(bronze_path_comments).show(5, truncate=False)

Bronze Layer sample:
+-----------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-------------------+------------------------+--------+---------+
|comment_key|text                                                                                                                                                                                                |date               |movie_id                |user_key|movie_key|
+-----------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-------------------+------------------------+--------+---------+
|23553      |Totam unde maxime rem. Laboriosam optio minima qui perferendis earum. Expedita maxime eum quo cumque ipsam. Unde expedita libero laboriosam e

In [115]:
import shutil
import os

if os.path.exists(silver_path_movies):
    shutil.rmtree(silver_path_movies)
if os.path.exists(silver_path_users):
    shutil.rmtree(silver_path_users)
spark.catalog.clearCache()


In [122]:
df_dim_movies.write.mode("overwrite").parquet(silver_path_movies)
df_dim_users.write.mode("overwrite").parquet(silver_path_users)

# Verification
print("Movies Dimension sample:")
spark.read.parquet(silver_path_movies).show(5, truncate=False)

print("Users Dimension sample:")
spark.read.parquet(silver_path_users).show(5, truncate=False)

Py4JJavaError: An error occurred while calling o935.parquet.
: org.apache.spark.SparkException: [FAILED_READ_FILE.FILE_NOT_EXIST] Encountered error while reading file file:///c:/Users/patri/Desktop/DS-2002-Final-Project/movie_data/dim_movies/part-00009-6e029371-8c38-4914-ac37-33502d8e4a84-c000.snappy.parquet. File does not exist. It is possible the underlying files have been updated.
You can explicitly invalidate the cache in Spark by running 'REFRESH TABLE tableName' command in SQL or by recreating the Dataset/DataFrame involved. SQLSTATE: KD001
	at org.apache.spark.sql.errors.QueryExecutionErrors$.fileNotExistError(QueryExecutionErrors.scala:831)
	at org.apache.spark.sql.execution.datasources.v2.FileDataSourceV2$.attachFilePath(FileDataSourceV2.scala:140)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.hasNext(FileScanRDD.scala:142)
	at org.apache.spark.sql.execution.FileSourceScanExec$$anon$1.hasNext(DataSourceScanExec.scala:695)
	at org.apache.spark.sql.cata

In [96]:
from pyspark.sql.types import StringType, StructType, StructField

# Cast all columns to string
df_comments = df_comments.astype(str)

# Define schema
comments_schema = StructType([
    StructField("comment_key", StringType(), True),
    StructField("comment_id", StringType(), True),
    StructField("text", StringType(), True),
    StructField("date", StringType(), True),
    StructField("movie_id", StringType(), True),
    StructField("user_key", StringType(), True),
    StructField("movie_key", StringType(), True)
])

# Create Spark DataFrame
df_comments_spark = spark.createDataFrame(df_comments, schema=comments_schema)


# Write Parquet
df_comments_spark.write.mode("overwrite").parquet(bronze_path_comments)

print(df_comments.shape)
print(df_comments.head())


df_bronze_check = spark.read.schema(comments_schema).parquet(bronze_path_comments)
df_bronze_check.show(5, truncate=False)


(41079, 7)
  comment_key                comment_id  \
0           1  5a9427648b0beebeb6957bda   
1           2  5a9427648b0beebeb6957a38   
2           3  5a9427648b0beebeb6957a4b   
3           4  5a9427648b0beebeb6957b89   
4           5  5a9427648b0beebeb6957a21   

                                                text                 date  \
0  Hic corporis illo adipisci similique. Omnis te...  2006-03-07 07:15:16   
1  Nobis incidunt ea tempore cupiditate sint. Ita...  2012-11-26 11:00:57   
2  Voluptatum voluptatem nam et accusamus ullam q...  2015-02-08 01:28:23   
3  Illo nihil occaecati minima facere ea nemo. Mo...  1976-12-18 08:14:46   
4  Minima odit officiis minima nam. Aspernatur id...  1981-11-08 04:32:25   

                   movie_id user_key movie_key  
0  573a1391f29313caabcd8978       65      1555  
1  573a1390f29313caabcd587d       29       656  
2  573a1390f29313caabcd5b9a       68      2296  
3  573a1391f29313caabcd82da       42      2637  
4  573a1390f29313caabc

In [75]:
bronze_query = (
    df_comments_bronze.writeStream
    .format("parquet")
    .outputMode("append")
    .option("checkpointLocation", os.path.join(checkpoint_path_comments, 'bronze'))
    .start(bronze_path_comments)
)

In [76]:
time.sleep(5)  # wait a few seconds for streaming to pick up files
df_bronze_check = spark.read.parquet(bronze_path_comments)
print("Bronze Layer Sample:")
df_bronze_check.show(5, truncate=False)
print(f"Bronze Layer Count: {df_bronze_check.count()}")

AnalysisException: Unable to infer schema for Parquet at . It must be specified manually.